<a href="https://colab.research.google.com/github/vitao-bolado/Trabalho1_AnaliseDados/blob/main/Trabalho1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Trabalho 1 - Analise de rotatividade de Clientes Bancários**<br>
**Desenvolvido por** José Vinicius e Vitor Faustino<Br>


*   Identicar os fatores que influenciam a saída dos clientes;
*   Comparar o perfil de clientes que saíram com os que permaneceram;
*   Desenvolver modelos preditivos de churn;
*   Trabalhar com variáveis categóricas e numéricas simultaneamente.


**ETAPA 1** - Carregando a base de dados

In [2]:
# 1. Introdução
print("Python é uma linguagem de programação de alto nível, interpretada e de tipagem dinâmica.")

var_1 = 2.3456
var_2 = 12

print(f"Valor de var_1: {var_1:.2f} \nValor de var_2: {var_2}")

Python é uma linguagem de programação de alto nível, interpretada e de tipagem dinâmica.
Valor de var_1: 2.35 
Valor de var_2: 12
